# Додаткова лабораторна: нтегрування з використанням CUDA, OpenAPI i OpenCl

Виконано командою:

* Бабенко Аліна (https://github.com/BabenkoAlina) 
* Бевз Євгеній (https://github.com/ybevz)

# Графік запуску програми на директорії №3

Загалом тенденція така сама, як і на попередні практичній, там теж запис займав набагато менше часу, ніж індектування файлів.<br>
Нижче наведено графік попередньої практичної

Код цієї практичної виконувався на 10 потоках індеквування та 10 потоках мердження. <br>

І по результатах паралельна програма випереджає аж у 10 разів, що є результатом кращим, ніж очікувалось

## §1: Аналіз результатів запусків з використанням профайлера

Тут ми розглянемо результати запуску програми на піддиректорії *3/0*. Запуск було виконано на *Linux Fedora 40*, використаний профайлер - *perf*. Аналогічно до попередньої лабораторної роботи з послідовним підрахунком слів, для отримання результатів профайлера виконуємо наступні кроки:
- `sudo perf -g ./bin/countwords_seq ./config.cfg`
- `sudo perf report`

Отримуємо наступний вивід:
![Perf Result](./results/perf_results.jpg)

Дивлячись на перші елементи секції *Children*, ми бачимо, що більшість часу програма перебуває у функціях, викликаних потоками, а також методом `WordsCounter::writeFileContentToMap`. Фактично ситуація дуже подібна до попередньої лабораторної, з єдинною відмінністю, що більшість роботи на себе взяли потоки.

## §2: Залежність швидкодії програми від кількості задіяних потоків

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import subprocess

cells, times = [[0]*10]*10, []

with open('./copy_config.cfg', "r") as file:
    cfg_content = file.readlines()

for i in range(1, 11):
    for j in range(1, 11):
        for k, line in enumerate(cfg_content):
            if line.startswith("indexing_threads ="):
                cfg_content[k] = f"indexing_threads = {i}\n"
            elif line.startswith("merging_threads  ="):
                cfg_content[k] = f"merging_threads  = {j}\n"
    
        with open('./copy_config.cfg', "w") as file:
            file.writelines(cfg_content)

        runner = subprocess.run(
            f"./bin/countwords_par ./copy_config.cfg",
            shell=True,
            capture_output=True,
            text=True,
            check=True
        )
        run_result = int(runner.stdout.splitlines()[0].replace("Total=", ''))
        cells[i-1][j-1] = run_result

times = np.array(cells)
x = np.arange(times.shape[0])  # Number of elements A
y = np.arange(times.shape[1])  # Number of elements B
X, Y = np.meshgrid(x, y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, times, cmap='viridis')

# Set labels
ax.set_xlabel('index threads')
ax.set_ylabel('merge threads')
ax.set_zlabel('time consumed')

plt.show()

# *Файли з результатами:*

lab7<br>
https://drive.google.com/drive/folders/14riOl1gT4sEkaWM-4-QVKmLyzA6QHSK3?usp=sharing<br>

lab8<br>
https://drive.google.com/drive/folders/1YZJVp-bzmeqRjh55KzVg2zuf4o66Swv3?usp=sharing<br>

Запуски виконувалися на повній директорії 3 і можна побачити, що результати збігаються, що і очікувалося